# DRRT star test

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from collections import Counter
import os
import pickle
import numpy as np

from shapely.geometry import Polygon

San check with one agent

In [ ]:
from swarm_prm.utils.agent_assignment import get_agent_assignment

num_agents = 1
agent_radius = 1

# Map config

map_type = "obstacle"
num_samples = 200



In [ ]:
map_fname = "{}_{}.pkl".format(map_type, num_samples)
fname = os.path.join("../maps", map_fname)
with open(fname, "rb") as f:
    gaussian_prm = pickle.load(f)



start_regions = [
    Polygon([[0, 160], [0, 0], [20, 0], [20, 160]]),
]

goal_regions = [
    Polygon([[180, 160], [180, 0], [200, 0], [200, 160]]),
]
num_starts = 1
num_goals = 1

starts_idx_set = gaussian_prm.get_node_index(start_regions)
goals_idx_set = gaussian_prm.get_node_index(goal_regions)

starts_idx = np.random.choice(starts_idx_set, num_starts, replace=False).tolist()
goals_idx = np.random.choice(goals_idx_set, num_goals, replace=False).tolist()

# Create weight pool

starts_pool = []
goals_pool = []

for start_idx in starts_idx:
    starts_pool += [start_idx] * gaussian_prm.gaussian_nodes[start_idx].get_capacity(agent_radius)

for goal_idx in goals_idx:
    goals_pool += [goal_idx] * gaussian_prm.gaussian_nodes[goal_idx].get_capacity(agent_radius)

start_per_agent = np.random.choice(starts_pool, num_agents, replace=False).tolist()
goal_per_agent = np.random.choice(goals_pool, num_agents, replace=False).tolist()

start_counts = Counter(start_per_agent)
goal_counts = Counter(goal_per_agent)

starts_idx = []
starts_agent_count = []
goals_idx = []
goals_agent_count = []

for start_idx, count in start_counts.items():
    starts_idx.append(start_idx)
    starts_agent_count.append(count)

for goal_idx, count in goal_counts.items():
    goals_idx.append(goal_idx)
    goals_agent_count.append(count)

print(starts_idx, goals_idx)
print(starts_agent_count, goals_agent_count)


In [ ]:
from swarm_prm.solvers.macro import SOLVER_REGISTRY

solver = SOLVER_REGISTRY["DRRTStarSolver"]

drrt_star_solver = solver(gaussian_prm, agent_radius, 
                    starts_agent_count=starts_agent_count, goals_agent_count=goals_agent_count,
                    starts_idx = starts_idx, goals_idx = goals_idx,
                   num_agents=num_agents, time_limit=120)
path, cost= drrt_star_solver.solve()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = gaussian_prm.visualize_roadmap()

for agent in range(num_agents):
    locations = [gaussian_prm.samples[location_idx[0]] for location_idx in p]
    x = [loc[0] for loc in locations]
    y = [loc[1] for loc in locations]
    ax.plot(x, y, "-")

plt.show()


Multi-agent DRRT

In [ ]:
num_agents = 4
agent_radius = 1

# Map config
map_type = "obstacle"
num_samples = 100

starts_weight = [0.2, 0.8]
goals_weight = [0.4, 0.6]

In [ ]:
# Load map and instance
import os
import pickle

from swarm_prm.utils.agent_assignment import get_agent_assignment

map_fname = "{}_{}.pkl".format(map_type, num_samples)
fname = os.path.join("../maps", map_fname)
with open(fname, "rb") as f:
    gaussian_prm = pickle.load(f)

starts_agent_count = get_agent_assignment(num_agents, starts_weight)
goals_agent_count = get_agent_assignment(num_agents, goals_weight)

print(starts_agent_count, goals_agent_count)

In [ ]:
from swarm_prm.solvers.macro import SOLVER_REGISTRY

solver = SOLVER_REGISTRY["DRRTStarSolver"]

drrt_star_solver = solver(gaussian_prm, agent_radius, starts_agent_count=starts_agent_count, goals_agent_count=goals_agent_count,
                   num_agents=num_agents, time_limit=300)
path, cost= drrt_star_solver.get_solution()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = gaussian_prm.visualize_roadmap()
for agent in range(num_agents):
    locations = [gaussian_prm.samples[location_idx[agent]] for location_idx in paths]

    x = [loc[0] for loc in locations]
    y = [loc[1] for loc in locations]
    ax.plot(x, y, "-")

plt.show()

In [ ]:
# Add interpolations 
import numpy as np

def add_linear_interpolation_points(path, subdiv=5):
    """
    Insert linearly-interpolated positions between consecutive points.
    
    Parameters
    ----------
    path : array-like of shape (N, 2)
        The original list/array of N 2D points, e.g. [(x1,y1), (x2,y2), ...].
    subdiv : int
        Number of intervals to subdivide each segment. 
        For each original segment, we will add 'subdiv - 1' new interior points 
        (except that we skip duplicates at the shared boundary).

    Returns
    -------
    new_path : np.ndarray of shape (M, 2)
        The new path including the original points and the added interpolation points.
    """
    path = np.asarray(path)
    new_path = []

    # Iterate over pairs of consecutive points
    for i in range(len(path) - 1):
        start = path[i]
        end = path[i + 1]
        
        # Create subdiv+1 points in [start, end] using linspace for each dimension
        xs = np.linspace(start[0], end[0], subdiv + 1)
        ys = np.linspace(start[1], end[1], subdiv + 1)
        
        # Add each intermediate point except the very last one
        # to avoid duplicating the next segment's start
        for j in range(subdiv):
            new_path.append([xs[j], ys[j]])

    # Finally add the very last point of the last segment
    new_path.append(path[-1].tolist())
    
    return np.array(new_path)

interpolated_paths = []
for p in simple_paths:
    new_path = add_linear_interpolation_points(p, 50)
    interpolated_paths.append(new_path)


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.animation import FuncAnimation

def animate_solution(fig, ax, speed, paths):
    """
        Visualize solution trajectory provided instance
    """
    
    agents = []
    def init():
        for agent in agents:
            agent.remove()
        agents.clear()
        return []

    def update(frame):
        idx = frame * speed
        for agent in agents:
            agent.remove()
        agents.clear()
        cmap = plt.get_cmap("tab10")
        locs = [path[idx] for path in paths]
        for i, loc in enumerate(locs):
            agent = ax.add_patch(Circle(loc, radius=agent_radius, color=cmap(i%10)))
            agents.append(agent)
        return agents

    anim = FuncAnimation(fig, update, frames=len(paths[0])// speed , 
                         init_func=init, blit=True, interval=10)
    anim.save("solutions/drrt_star_solution.gif", writer='pillow', fps=6)

# fig, ax = instance.visualize()
fig, ax = gaussian_prm.visualize_g_nodes()
animate_solution(fig, ax, 10, interpolated_paths)
